## Star Tracker ML Pipeline ##

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


### Filepaths and Hyperparameters ###

In [5]:
# Data filepath for training
file = "../images_data/mag5_1608_no_adverserial_gray/bins/mag5_1608_bin_features.csv"

# Configuration values
validation_split = 0.20
num_epochs = 50
batch_size = 40
saved_model_dir = 'models/'

In [7]:

# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(file, header=0)
input_size = len(dataframe.columns) - 1
dataset = dataframe.values
X = dataset[:,1:(input_size+1)].astype(int)
Y = dataset[:,0].astype(int)
print("Input Size:", len(X[0]))
num_classes = len(np.unique(Y))
print("Output Size:", num_classes)

Input Size: 10
Output Size: 1608


In [8]:
# Encode class values as integers and then create dataframe with one hot encoding
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_Y = tf.one_hot(encoded_Y, depth=num_classes)
print(dummy_Y)



# Unused code from when I was sanity checking
"""
for i in range(100):
    if not (encoder.classes_[dummy_Y.numpy()[i].argmax()] == Y[i]):
        print("Error")
        # print(dummy_Y.numpy()[i].argmax())
        # print(Y[i])
        # print(encoder.classes_[dummy_Y.numpy()[i].argmax()])
"""

"""
# star_labels = np.zeros(num_classes).astype(int)
# get labels in num_classes sized list

# one_hot_array = dummy_Y.numpy()
# # print(Y[0])
# for i in range(len(one_hot_array)):
#     if star_labels[one_hot_array[i].argmax()] == 0:
#         star_labels[one_hot_array[i].argmax()] = Y[i]
# np.set_printoptions(threshold=np.inf)
# print(star_labels)
"""


2024-07-03 14:22:18.782577: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 14:22:18.996609: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 14:22:18.996705: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 14:22:19.000312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 14:22:19.000421: I external/local_xla/xla/stream_executor

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(49848, 1608), dtype=float32)


'\n# star_labels = np.zeros(num_classes).astype(int)\n# get labels in num_classes sized list\n\n# one_hot_array = dummy_Y.numpy()\n# # print(Y[0])\n# for i in range(len(one_hot_array)):\n#     if star_labels[one_hot_array[i].argmax()] == 0:\n#         star_labels[one_hot_array[i].argmax()] = Y[i]\n# np.set_printoptions(threshold=np.inf)\n# print(star_labels)\n'

In [9]:
# Create the ML Model
star_model = tf.keras.Sequential([
  layers.Dense(input_size, activation="relu", name="layer1"),
  layers.Dense(64, activation="relu", name="layer2"),
  layers.Dense(32, activation="relu", name="layer3"),
  layers.Dense(1608, activation="softmax", name ="layer4")
])

star_model.compile(loss = "categorical_crossentropy",
                      optimizer = 'adam', metrics = ['accuracy'])

In [10]:
# Model Training
star_model.fit(X, dummy_Y, epochs=num_epochs)

Epoch 1/50


I0000 00:00:1720041762.711417   62835 service.cc:145] XLA service 0x7f4018006160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720041762.711475   62835 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-07-03 14:22:42.754892: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-03 14:22:42.929767: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720041764.056365   63050 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 48 bytes spill stores, 48 bytes spill loads

I0000 00:00:1720041764.085204   63048 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 304 bytes spill stores, 304 bytes spill loads


 119/1558 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 5.5558e-04 - loss: 7.3668

I0000 00:00:1720041765.202701   62835 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1538/1558 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1451 - loss: 5.1016

I0000 00:00:1720041768.178600   63171 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1720041768.267901   63173 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 300 bytes spill stores, 300 bytes spill loads



1558/1558 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.1476 - loss: 5.0765
Epoch 2/50
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7225 - loss: 0.9246
Epoch 3/50
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8616 - loss: 0.4725
Epoch 4/50
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9013 - loss: 0.3368
Epoch 5/50
1486/1558 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9233 - loss: 0.2719

KeyboardInterrupt: 

In [30]:
# Save Model

star_model.save(saved_model_dir + 'star_tracker_model_large.keras')
if not os.path.exists(saved_model_dir + 'star_tracker_saved_model/'):
    os.makedirs(saved_model_dir + 'star_tracker_saved_model/')
star_model.export(saved_model_dir + 'star_tracker_saved_model/')

# TODO: tflite conversion is not working.
# # Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir + 'star_tracker_saved_model/') # path to the SavedModel directory
tflite_model = converter.convert()

# # Save the model.
with open(saved_model_dir + 'star_tracker_model_small.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: models/star_tracker_saved_model/assets


INFO:tensorflow:Assets written to: models/star_tracker_saved_model/assets


Saved artifact at 'models/star_tracker_saved_model/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1608), dtype=tf.float32, name=None)
Captures:
  140468450171136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470127857152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470127856976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243647216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243655312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243651440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243660592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243661648: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1719977882.345787  113281 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1719977882.345828  113281 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-07-02 20:38:02.346039: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/star_tracker_saved_model/
2024-07-02 20:38:02.346465: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-02 20:38:02.346472: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/star_tracker_saved_model/
2024-07-02 20:38:02.351013: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-07-02 20:38:02.369465: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: models/star_tracker_saved_model/
2024-07-02 20:38:02.374925: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for tags { serve }; Status: success: OK. Took 28888 microsecond